In [ ]:
import pandas as pd
from pandas import DataFrame
import portfolio
from stats import *
from visualisation import *

In [ ]:
# Read in the pre-processed data
ticker = "AAPL"
data_path = f"../data/{ticker}_processed_hourly_data.csv"
data = pd.read_csv(data_path, index_col='Datetime', parse_dates=True)

In [ ]:
def calculate_bollinger_bands(df, window=50, num_std=2):
    rolling_mean = df['Close'].rolling(window).mean() 
    rolling_std = df['Close'].rolling(window).std()

    df['Bollinger_Upper'] = rolling_mean + (rolling_std * num_std)
    df['Bollinger_Lower'] = rolling_mean - (rolling_std * num_std)

In [ ]:

# Calculate the Bollinger bands
calculate_bollinger_bands(data)

# Filter out the rows that will be null
data = data[(data['Bollinger_Upper'].notna()) & (data['Bollinger_Lower'].notna())]

# Set up Portfolio
portfolio = portfolio.Portfolio(initial_cash=100000)
data['Portfolio Value'] = None
data['Actions'] = None

stop_loss_percentage = 0.02
take_profit_percentage = 0.1

# Time period (for days set it to 252 and for hourly set to 1638)
time_period = 1638

In [ ]:
for index, row in data.iterrows():
    data.at[index, 'Portfolio Value'] = portfolio.get_value(row['Close'])

    # Stop loss and take profit logic here
    if portfolio.holdings and portfolio.last_buy_price:
        if row['Close'] < (portfolio.last_buy_price * (1 + take_profit_percentage)):
            portfolio.sell(row['Close'], portfolio.holdings)
            data.at[index, 'Actions'] = -1
        elif row['Close'] >  (portfolio.last_buy_price * (1 - stop_loss_percentage)):
            portfolio.sell(row['Close'], portfolio.holdings)
            data.at[index, 'Actions'] = -1

    # Mean reversion logic here
    if row['Close'] < row['Bollinger_Lower'] and not portfolio.holdings:
        # Buy when price falls below the lower Bollinger Band
        portfolio.buy(row['Close'], 250)
        data.at[index, 'Actions'] = 1

    elif row['Close'] > row['Bollinger_Upper'] and portfolio.holdings:
        # Sell when price rises above the upper Bollinger Band
        portfolio.sell(row['Close'], 250)
        data.at[index, 'Actions'] = -1


# Calculate the percentage change (for analysis)
data['Returns'] = data['Portfolio Value'].pct_change().dropna()

print(f"Final Portfolio Value: {portfolio.get_value(data.iloc[-1]['Close'])}")

In [ ]:
years = ((data.index[-1] - data.index[0]).days / 365)

cagr = calculate_compound_annual_growth_rate(data.iloc[0]['Portfolio Value'], data.iloc[-1]['Portfolio Value'], years)
sharpe_ratio = calculate_sharpe_ratio(data['Returns'], periods_per_year=time_period)
max_drawdown = calculate_max_drawdown(data['Portfolio Value'].dropna().tolist())
volatility = calculate_volatility(data['Returns'], periods_per_year=time_period)

In [ ]:
print(f"Compound Annual Growth Rate: {cagr}")
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"max_drawdown: {max_drawdown}")
print(f"volatility: {volatility}")

In [ ]:
plot_portfolio_value(data, "Mean Reversion")

In [ ]:
plot_trade_points(data, "Mean Reversion")